In [248]:
import pandas as pd
import numpy as np
from jina import Document, DocumentArray, Flow, Executor, requests 

In [249]:
df_emb = pd.read_csv("/Users/peppermint/Desktop/codes/python/cybersecurity-jina/data/embeddings_df.csv")
print(len(df_emb))
df_emb.head(2)

15101


,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,Label
15099,117728350.0,0.0,0.0,403577730.0,0.0,0.0,295222100.0,0.0,0.0,179444350.0,...,0.0,0.0,188270500.0,0.0,0.0,0.0,87271100.0,0.0,0.0,1.0
15100,117723870.0,0.0,0.0,403589820.0,0.0,0.0,295222200.0,0.0,0.0,179451800.0,...,0.0,0.0,188267360.0,0.0,0.0,0.0,87276080.0,0.0,0.0,1.0


In [ ]:
df_ = pd.read_csv("/Users/peppermint/Desktop/codes/python/cybersecurity-jina/data/embeddings_df.csv")
print(len(df_emb))
df_emb.head(2)

## Resample data so all the attacks arent at the bottom

In [242]:
df_res = df_res.sample(frac=1, random_state=0)
df_res.tail(2)

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
15099,80,6,1.519355e+09,313,2,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,SQL Injection
15100,80,6,1.519383e+09,190,2,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,SQL Injection


## Join embeddings with details from non-embedded data

In [244]:
res = pd.concat([df_emb, df_res], axis=1)
res.head()

,0,1,2,3,4,5,6,7,8,9,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,117520104.0,0.0,0.0,403782720.0,0.0,0.0,295003260.0,0.0,0.0,179647220.0,...,20,0.0,0.000000,0,0,0.0,0.000000,0,0,Benign
1,117521950.0,0.0,0.0,403168480.0,0.0,0.0,295557760.0,0.0,0.0,178901280.0,...,20,0.0,0.000000,0,0,0.0,0.000000,0,0,Benign
2,38913344.0,0.0,0.0,481586300.0,0.0,0.0,156842260.0,0.0,30670994.0,219856660.0,...,20,79440.5,25166.637449,97236,61645,58533700.5,591071.265501,58951651,58115750,Benign
3,117017290.0,0.0,0.0,404279580.0,0.0,0.0,294600860.0,0.0,0.0,180125760.0,...,20,0.0,0.000000,0,0,0.0,0.000000,0,0,Benign
4,116702550.0,0.0,0.0,404589600.0,0.0,0.0,294278500.0,0.0,0.0,180431680.0,...,20,0.0,0.000000,0,0,0.0,0.000000,0,0,Benign


In [245]:
embs = res.iloc[:, :128]
embs.tail(2)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
15099,117728350.0,0.0,0.0,403577730.0,0.0,0.0,295222100.0,0.0,0.0,179444350.0,...,174537500.0,0.0,0.0,188270500.0,0.0,0.0,0.0,87271100.0,0.0,0.0
15100,117723870.0,0.0,0.0,403589820.0,0.0,0.0,295222200.0,0.0,0.0,179451800.0,...,174543660.0,0.0,0.0,188267360.0,0.0,0.0,0.0,87276080.0,0.0,0.0


In [246]:
detailed_embs = res.iloc[:, 128:]
detailed_embs.head(2)


,Label,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,52412,6,1.519359e+09,46,2,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1,0.0,52416,6,1.519359e+09,1,2,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign


## Now that we have "scrambled" the data, add mock date/time column so that we can simulate this data being realtime later in streamlit and make aggregation of statistics easy in dashboard

In [247]:
def random_dates(start, end, n, unit='H', seed=None):
    if not seed:  # from piR's answer
        np.random.seed(0)

    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.rand(n) * ndays, unit=unit) + start

np.random.seed(0)
start = pd.to_datetime('2022-01-03')
end = pd.to_datetime('2022-01-10')
dt_idx = random_dates(start, end, len(df_res))
dt_idx

DatetimeIndex(['2022-01-03 04:23:25.828911600',
               '2022-01-03 05:43:17.453751600',
               '2022-01-03 04:49:19.585232400',
               '2022-01-03 04:21:32.635670400',
                  '2022-01-03 03:23:21.258222',
                  '2022-01-03 05:10:01.750458',
                  '2022-01-03 03:30:02.511684',
               '2022-01-03 07:08:03.062421600',
               '2022-01-03 07:42:33.487502400',
               '2022-01-03 03:04:03.115743600',
               ...
               '2022-01-03 01:59:32.601598800',
               '2022-01-03 01:35:38.429199599',
               '2022-01-03 06:16:14.548357200',
               '2022-01-03 01:20:37.308991200',
               '2022-01-03 06:57:30.496557600',
               '2022-01-03 05:08:51.639637200',
               '2022-01-03 06:57:41.524484400',
               '2022-01-03 07:50:45.328786800',
               '2022-01-03 05:20:55.199824800',
               '2022-01-03 07:08:20.982969600'],
              dtype=

cols = cols[-1:] + cols[:-1]